In [2]:
import numpy as np
import cmath

# Parameters
k = 1
R = 0.
alpha = 0.9
T = 0.05
Nr = 100
Nt = 500
Bo = 0.1
Ca = 0.1
D = 0.

# Discretization
dr = (1 - alpha) / (Nr - 1)
dt = T / Nt

# Known functions

def W_0(r):
    return 1/4*(2*np.log(r/alpha) - (r**2 - alpha**2))
def dW_0(x): 
    return 1/2*(1/r - r)
def d2W_0(x):
    return -1/2*(1/r**2 + 1)


# Grid initialization
r = np.linspace(alpha, 1, Nr)
t = np.linspace(0, T, Nt)
psi = np.zeros((Nr, Nt), dtype=complex)

# Initial condition
psi[:, 0] = np.cos(10*r)


In [3]:
# Time-stepping loop
for n in range(0, Nt-1):
    # Update interior points using the discretized equation
    for i in range(2, Nr-2):
        # Compute the derivatives
        psi_r = (psi[i+1, n] - psi[i-1, n]) / (2*dr)
        psi_rr = (psi[i+1, n] - 2*psi[i, n] + psi[i-1, n]) / dr**2
        psi_rrr = (psi[i+2, n] - 2*psi[i+1, n] + 2*psi[i-1, n] - psi[i-2, n]) / (2*dr**3)
        psi_rrrr = (psi[i+2, n] - 4*psi[i+1, n] + 6*psi[i, n] - 4*psi[i-1, n] + psi[i-2, n]) / dr**4
                
        # Discretize the time derivatives using forward differences (explicit)
        psi_t = (psi[i, n+1] - psi[i, n]) / dt
        psi_rt = (psi[i+1, n+1] - psi[i-1, n+1] - psi[i+1, n] + psi[i-1, n]) / (2*dr*dt)
        psi_rrt = (psi[i+1, n+1] - 2*psi[i, n+1] + psi[i-1, n+1] - psi[i+1, n] + 2*psi[i, n] - psi[i-1, n]) / (dr**2*dt)

        # Construct the discretized equation
        LHS = (psi_rrrr * r[i]**3 - 2 * psi_rrr * r[i]**2 - psi_rrt * R * r[i]**3 +(-1j * W_0(r[i]) * R * k * r[i]**3 - 2 * k**2 * r[i]**3 + 3 * r[i]) * psi_rr +psi_rt * R * r[i]**2 + 1j * psi[i, n] * d2W_0(r[i]) * R * k * r[i]**3 +(1j * R * k * r[i]**2 * W_0(r[i]) + 2 * k**2 * r[i]**2 - 3) * psi_r +(R * k * r[i] * psi_t + (-1j * R * dW_0(r[i]) + (1j * R * W_0(r[i]) + k) * k**2 * r[i]) * psi[i, n]) * k * r[i]**2) / r[i]**4
                
        # Update the solution using the explicit method
        psi[i, n+1] = psi[i, n] - LHS * dt

    # Apply the boundary conditions
        # Note: You'll need to discretize the given boundary conditions and implement them here.
        # Make sure to use the appropriate discretization schemes for the mixed derivatives.
            # Apply the boundary conditions
            # At r = alpha
        psi[0, n+1] = 0
        psi[1, n+1] = psi[2, n+1]  # Enforce d(psi)/dr = 0 using a forward difference approximation

            # At r = 1
            # Note: You'll need to discretize the given boundary conditions and implement them here.
            # Make sure to use the appropriate discretization schemes for the mixed derivatives.

        # First boundary condition at r = 1
        psi_rrt_1 = (psi[-1, n+1] - 2*psi[-1, n] + psi[-1, n-1]) / (dr**2*dt) - (psi[-2, n+1] - 2*psi[-2, n] + psi[-2, n-1]) / (dr**2*dt)
        psi_rt_1 = (psi[-1, n+1] - psi[-1, n]) / dt - (psi[-2, n+1] - psi[-2, n]) / (dt*dr)
        psi_r_1 = (psi[-1, n] - psi[-2, n]) / dr
        psi_rr_1 = (psi[-1, n] - 2*psi[-2, n] + psi[-3, n]) / dr**2
        psi_t_1 = (psi[-1, n+1] - psi[-1, n]) / dt

        BC1 = (psi_rt_1 / r[-1]**2 - psi_rrt_1 / r[-1] - k**2 * psi_t_1 / r[-1] + 1j * k * W_0(r[-1]) * (psi_r_1 / r[-1]**2 - psi_rr_1 / r[-1] - k**2 * psi[-1, n] / r[-1]) - 1j * k * psi[-1, n] / r[-1])
        
        # Second boundary condition at r = 1
        psi_r_1 = (psi[-1, n] - psi[-2, n]) / dr
        psi_rr_1 = (psi[-1, n] - 2*psi[-2, n] + psi[-3, n]) / dr**2
        psi_rrr_1 = (psi[-1, n] - 2*psi[-2, n] + psi[-3, n] - psi[-2, n] + 2*psi[-3, n] - psi[-4, n]) / (2*dr**3)
        psi_rt_1 = (psi[-1, n+1] - psi[-1, n]) / dt - (psi[-2, n+1] - psi[-2, n]) / (dt*dr)

        BC2 = (-1j * Bo * (1j * (psi_r_1 * W_0(r[-1]) * R * k * r[-1]**2 - psi[-1, n] * dW_0(r[-1]) * R * k * r[-1]**2 + k**2 * psi_r_1 * r[-1]**2) + psi_rt_1 * R * r[-1]**2 - psi_rrr_1 * r[-1]**2 + psi_rr_1 * r[-1] - psi_r_1)) / (r[-1]**3 * k) - 2 * Ca * (-1j * k * psi[-1, n] / r[-1]**2 + 1j * k * psi_r_1 / r[-1]) + (k**2 - D) * (psi_r_1 / r[-1]**2 - psi_rr_1 / r[-1] - k**2 * psi[-1, n] / r[-1])


        # Update the solution at r = 1 based on the boundary conditions
        psi[-1, n+1] = psi[-2, n+1] - dr * (BC1 + BC2) / 2



TypeError: only length-1 arrays can be converted to Python scalars